In [1]:
import glob
import re
import json
import pandas as pd

basepath = '/home/xilini/tell-me-more-coref/data/conllua-processed/'

In [2]:
parsed_files = glob.glob(basepath + '*.csv')

In [3]:
len(parsed_files)

537

In [4]:
with open('../../../../../data/tmm_bugged_annotations.json', 'r') as bf:
    bugged_files = json.load(bf)

In [5]:
toremove = []
for file in bugged_files:
    fileid = file.split('/')[-1].split('.conllua')[0]
    annotator = file.split('/')[-3]
    if annotator == 'annotator1':
        annotator_type = 'ann1'
    elif annotator == 'annotator2':
        annotator_type = 'ann2'
    filename = f'{basepath}/{annotator_type}-{fileid}.csv'
    toremove.append(filename)
for elem in parsed_files:
    if elem in toremove:
        print(elem)
parsed_files = [elem for elem in parsed_files if elem not in toremove]

In [6]:
len(parsed_files)

537

In [7]:
import tqdm

tmm_texts = []

for file in tqdm.tqdm(parsed_files):

    df = pd.read_csv(file)

    sentences = []
    current_sentence = []
    for form in df['FORM']:
        if pd.isna(form):
            if current_sentence:
                sentences.append(' '.join(current_sentence))
                current_sentence = []
        else:
            current_sentence.append(form)
    if current_sentence:
        sentences.append(' '.join(current_sentence))

    #print(sentences)
    
    processed_data = []
    for i, sentence in enumerate(sentences, 1):
        words = sentence.split()
        tokens = [{'id': idx+1, 'text': word} for idx, word in enumerate(words)]
        processed_data.append({
            'id': i,
            'speaker': None,
            'text': sentence,
            'tokens': tokens
        })

    text_dict = {}
    text_dict['id'] = 'doc_' + str(file.split('/')[-1].split('.csv')[0])
    text_dict['sentences'] = processed_data
    text_dict['coref_chains'] = None
    
    tmm_texts.append(text_dict)

100%|██████████████████████████████████████████████| 537/537 [00:00<00:00, 1182.20it/s]


In [8]:
import json
with open('/home/xilini/tell-me-more-coref/run1/inputs_tmm_537.json', 'w') as f:
    json.dump(tmm_texts, f)